In [ ]:
!pip install flask line-bot-sdk

In [ ]:
import json
from random import choices, randint
from copy import deepcopy
from urllib.parse import parse_qs
from time import strftime

from flask import Flask, request, abort
from pymongo.mongo_client import MongoClient
import pymongo

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage,
    FlexMessage,
    FlexContainer
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    PostbackEvent,
    FollowEvent
)

app = Flask(__name__)

with open('env.json') as f:
    env = json.load(f)
configuration = Configuration(access_token=env['YOUR_CHANNEL_ACCESS_TOKEN'])
handler = WebhookHandler(env['YOUR_CHANNEL_SECRET'])
with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

try:
    uri = 'your_mongodb'
    client = MongoClient(uri)
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
    db = client['linebot']
    users = db['users']  # collection
except Exception as e:
    print(e)

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    line_bot_api.reply_message_with_http_info(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=event.message.text)]
        )
    )

        
@handler.add(PostbackEvent)
def handle_message(event):
    msg = None
    nav_a = 'your_richmenu'
    nav_l = 'your_richmenu'
    nav_r = 'your_richmenu'
    
    qs = parse_qs(event.postback.data)
    if qs.get('fn') == ['donav']:
        u = get_user(0)
        p = line_bot_api.get_profile(u['user_id'])
        msg = FlexMessage(altText='fans', contents=FlexContainer.from_dict(fan(p)))
        line_bot_api.link_rich_menu_id_to_user(user_id=event.source.user_id,
                                               rich_menu_id=nav_r)
        set_pos(event.source.user_id, 0)

    elif qs.get('fn') == ['nav']:  # fn=nav&dir=LL
        d = qs.get('dir')
        if d and d[0] and d[0] in ['LL', 'L', 'R', 'RR']:
            pos = get_pos(event.source.user_id)
            amount = users.count_documents({})
            new = {'LL': 0, 'L': pos-1, 'R': pos+1, 'RR': amount-1}[d[0]]
            new = max(min(new, amount-1), 0)
            u = get_user(new)
            if u:
                p = line_bot_api.get_profile(u['user_id'])
                msg = FlexMessage(altText='fans', contents=FlexContainer.from_dict(fan(p)))
                set_pos(event.source.user_id, new)
                if new == 0:
                    line_bot_api.link_rich_menu_id_to_user(user_id=event.source.user_id,
                                                           rich_menu_id=nav_r)
                elif new == amount-1:
                    line_bot_api.link_rich_menu_id_to_user(user_id=event.source.user_id,
                                                           rich_menu_id=nav_l)
                else:
                    line_bot_api.link_rich_menu_id_to_user(user_id=event.source.user_id,
                                                           rich_menu_id=nav_a)
            
    elif qs.get('fn') == ['exitnav']:
        line_bot_api.unlink_rich_menu_id_from_user(event.source.user_id)

    if msg:
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[msg]
            )
        )

        
@handler.add(FollowEvent)
def handle_follow(event):
    profile = line_bot_api.get_profile(event.source.user_id)
    follow(event.source.user_id, profile)
    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[TextMessage(text=f'Welcome {profile.display_name}')]
        )
    )

def get_pos(user_id):
    u = users.find_one({"_id": user_id})
    return u.get('nav_pos', 0) if u else 0


def get_user(pos):
    us = list(users.find().sort("follow", pymongo.DESCENDING).skip(pos).limit(1))
    return us[0] if (us and us[0]) else None


def set_pos(user_id, pos):
    if users.find_one({"_id": user_id}):
        u = {'nav_pos': pos}
        users.update_one({'_id': user_id}, {'$set': u})
        return True
    return False


def fan(profile):
    with open('meactions.json', encoding='utf-8') as f:
        bubble = json.loads(f.read())
    b = deepcopy(bubble)
    b["hero"]["url"] = profile.picture_url
    b["body"]["contents"][0]["contents"][1]["text"] = profile.user_id
    b["body"]["contents"][1]["contents"][1]["text"] = profile.display_name or '?'
    b["body"]["contents"][2]["contents"][1]["text"] = profile.status_message or '?'
    b["footer"]["contents"][0]["action"]["uri"] = profile.picture_url
    b["footer"]["contents"][1]["action"]["data"] = profile.user_id
    return b
    

def follow(user_id, profile):
    if users.find_one({'_id': user_id}):
        u = {'follow': strftime('%Y/%m/%d-%H:%M:%S')}
        u.update(dict(profile))
        users.update_one({'_id': user_id}, {'$set': u})
    else:
        u = {'_id': user_id,
             'follow': strftime('%Y/%m/%d-%H:%M:%S'),
             'unfollow': None}
        u.update(dict(profile))
        users.insert_one(u)  
    
    
if __name__ == "__main__":
    app.run()